In [1]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [3]:
# Preparing the training set and the test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')


In [4]:
# print (training_set.describe())
training_set.tail()
# user id | item id | rating | timestamp. 

,1,1.1,5,874965758
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275
79998,943,1330,3,888692465


In [5]:
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')

In [6]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))



In [7]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [8]:
print (len(training_set))
print (len(training_set[0]))
print (sum(training_set[30]))
print (training_set[0])

943
1682
90.0
[0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 1.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 5.0, 0.0, 2.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0, 2.0, 5.0, 4.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 2.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 0.0, 5.0, 1.0, 5.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 4.0, 5.0, 0.0, 2.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 4.0, 3.0, 5.0, 1.0, 3.0, 0.0, 3.0, 2.0, 0.0, 4.0, 0.0, 4.0, 3.0, 0.0, 2.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 5.0, 2.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 5.0, 3.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 5.0, 0.0, 5.0, 

In [9]:
print (len(test_set))
print (len(test_set[0]))
print (sum(test_set[30]))
print (test_set[0])

943
1682
51.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 4.0, 0.0, 3.0, 3.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 3.0, 0.0, 5.0, 4.0, 0.0, 3.0, 0.0, 3.0, 3.0, 0.0, 4.0, 3.0, 1.0, 0.0, 4.0, 0.0, 1.0, 0.0, 4.0, 5.0, 5.0, 0.0, 4.0, 3.0, 5.0, 0.0, 0.0, 0.0, 4.0, 5.0, 3.0, 0.0, 0.0, 0.0, 5.0, 3.0, 4.0, 0.0, 5.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 1.0, 5.0, 5.0, 0.0, 0.0, 3.0, 3.0, 0.0, 1.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 5.0, 3.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 5.0, 4.0, 0.0, 0.0, 5.0, 2.0, 0.0, 4.0, 0.0, 3.0, 4.0, 4.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 4.0, 4.0, 4.0, 0.0, 3.0, 3.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 

In [10]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [21]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    
#     default init method
    def __init__(self, ):
#         let's get super class init method to inherit all classes and methods from super class
        super(SAE, self).__init__()
#     Now, Let's make the architecture
# Input layer, first full connection is associated with our self class object
        self.fc1 = nn.Linear(nb_movies, 20)
#     Hidden layer, second full connection
        self.fc2 = nn.Linear(20, 10)
#     Hidden layer, third full connection for decoding
        self.fc3 = nn.Linear(10, 20)
#     Output layer, fourth full connection
        self.fc4 = nn.Linear(20, nb_movies)
#     let's put activation function on output layer
        self.activation = nn.Sigmoid()

#         forward propogation method
    def forward(self, x):
        # we need self here to make sure that all the function we need
#         should be comiing from local class which is our SME instead of super class nn module
# Encoding
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
#         decoding
        x = self.activation(self.fc3(x))
#     decode without activation funcion
        x = self.fc4(x)
        return x


In [ ]:
# Let's now get object from our Stack Auto Encoder SAE
sae = SAE()
# Let's now define criterion for the loss function
criterion = nn.MSELoss()
# Let's now define the optimizer and set hyper parameters
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


In [12]:
# Training the SAE
nb_epoch = 100
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            if id_user == 30:
                accuracy = torch.eq(target.data, torch.round(output.data))
                print ("Total Match:", torch.sum(accuracy))
                print ("Accuracy:", torch.sum(accuracy)/len(training_set[0]))
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


Total Match: 1659
Accuracy: 0.9863258026159334
epoch: 1 loss: 1.7715352492
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 2 loss: 1.09653719344
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 3 loss: 1.05351528082
Total Match: 1665
Accuracy: 0.9898929845422116
epoch: 4 loss: 1.03836975746
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 5 loss: 1.03102962957
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 6 loss: 1.02644385161
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 7 loss: 1.0237335148
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 8 loss: 1.02174260081
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 9 loss: 1.02081670764
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 10 loss: 1.01990683902
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 11 loss: 1.01889266976
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 12 loss: 1.01828986685
Total Match: 1666
Accuracy: 0.990487514863258
epoch: 13 loss: 1.01774414293
Total Match: 1666
Acc

In [16]:
print (len(test_set))
print (len(test_set[0]))
# print (sum(test_set[30]))
# data[:,1][data[:,0] == id_users]
print ("Total 0.0 ratings out of all 1682 ratings are:", len((test_set[0][test_set[0]==0.0])))
print (test_set[0])

943
1682
Total 0.0 ratings out of all 1682 ratings are: 1546

 0
 0
 0
⋮ 
 0
 0
 0
[torch.FloatTensor of size 1682]



In [17]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        if id_user == 30:
            accuracy = torch.eq(target.data, torch.round(output.data))
            print ("Total Match:", torch.sum(accuracy))
            print ("Accuracy:", torch.sum(accuracy)/len(training_set[0]))
            number_30 = torch.round(output.data)
            print ("Total 0.0 ratings out of all ratings:", len((number_30[number_30 == 0.0])))
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

Total Match: 1674
Accuracy: 0.995243757431629
Total 0.0 ratings out of all ratings: 1669
test loss: 0.959970077971


In [18]:
# let's subtract # of 0.0 ratings from total
print ("User 30 give ratings to these number of movies:")
1682 - 1546

User 30 give ratings to these number of movies:


136

In [19]:
# let's subtract # of 0.0 ratings from total
print ("and then we predicted these total ratings for user 30:")
1682 - 1669

and then we predicted these total ratings for user 30:


13